#### CARGA DA BIBLIOTECA

In [4]:
import re
import string
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
# spacy for lemmatization
import spacy
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

%matplotlib inline

import warnings
import webbrowser

from gensim.models.ldamulticore import LdaMulticore
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

In [5]:
# OLX1 = pd.read_csv("Base_OLX_1.csv", sep ='|', encoding = 'utf-8')

path = './data/04_feat_eng_df/'

# Nome dos arquivos dentro do diretorio acima (raw_df)
filenames = os.listdir(path)

# Dataframe dos dados sem tratamento
OLX1 = pd.read_csv(path + filenames[-2], sep='|') # -1 para pegar o ultimo dataframe adquirido

#### IMPORTAR STOP WORD

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('portuguese')
palavras_comuns = ['apto', 'apartamento', 'dorm', 'dorms', 'sala', 'sendo', 'cidade', 'vila', 'cresci', 'contato','alugar', 'www', 'anuncio', 'agende',\
                   'visita', 'fotos', 'foto', 'ver', 'numero', 'connheca', 'caracteristicas', 'caracteristica', 'corretora','ap', 'residencial', 'deste',\
                   'desta', 'facil', 'possui', 'hoje', 'imovel', 'vender', 'venda', 'alugar', 'informacoes', 'sobre', 'locacao', 'agora', 'adquirir','conheca'\
                   , 'conhecer', 'venha', 'atendimento', 'cliente', 'consultoria', 'imobiliaria', 'ltda', 'ref', 'atualizado', 'm'+chr(178), 'corretor', 'corretora', 'creci','immovel', 'imoveis']
stop_words = stop_words + palavras_comuns


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### ABRIR ARQUIVOS CARREGADOS NO BANCO DE DADOS

#### CRIA FAIXA DE VALORES E TOTAIS

In [10]:
## Comentei tudo abaixo pois essa feature ja foi tratada no notebook 03_limpeza_de_dados
# val1 = 0
# cond = []
# OLX1['condominio'].fillna(0, inplace = True)
# for index, row in OLX1.iterrows():
#     if row['condominio'] != 0:
#         val1 = int(row['condominio'].replace('.','',).split('$ ')[1])
#     else:
#         val1 = int(row['condominio'])
    
#     cond.append(val1)
# OLX1['condominio'] = cond


val2 = 0
iptu1 = []
OLX1['iptu'].fillna(0, inplace = True)
for index, row in OLX1.iterrows():
    if row['iptu'] != 0:
        val2 = int(row['iptu'].replace('.','',).split('$ ')[1])
    else:
        val2 = int(row['iptu'])
    
    iptu1.append(val2)
OLX1['iptu'] = iptu1

OLX1['TOTAL'] = OLX1['iptu'] + OLX1['condominio'] + OLX1['price']
OLX1['FAIXAS'] = pd.cut(OLX1.TOTAL, [0,500,1000, 1500, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 30000, 40000, 100000], labels = ['0 - 500','500-1000', '1000-1500', '1500-2000', '2000-3000', '3000-4000','4000-5000', '5000-6000','6000-7000', '7000-8000', '8000-9000' , '9000-10000','10000-20000', '20000-30000','30000-40000', 'maior que 40000'], include_lowest=True)




#### UNIFICA FEATURES TABULADAS E ARMAZENA NA DESC_3

In [11]:
desc_1 = []
desc_1b = []
OLX1['apartment_features'].fillna('', inplace = True)
OLX1['apartment_complex_features'].fillna('', inplace = True)
OLX1['garage_spaces'].fillna(0, inplace = True)

for index, row in OLX1.iterrows():
    if (row['apartment_complex_features'] != ''):
        desc_1.append (str(row['apartment_features']).replace(' ','_').replace(',_',', ')+ ', '+ str(row['apartment_complex_features']).replace(' ','_').replace(',_',',')\
        + ', ' + str(row['rooms'])+'_quarto'\
        + ', ' + str(row['rooms'])+'_quarto'\
        + ', ' + str(row['size'])+'_area'\
        + ', ' + str(row['size'])+'_area'\
        + ', ' + str(row['bathrooms'])+'_banheiro'\
        + ', ' + str(row['garage_spaces'])+'_vaga'\
        + ', ' + str(row['FAIXAS'])\
        + ', ' + str(row['FAIXAS'])\
        + ', ' + str(row['FAIXAS']))
        
                  
    else:
        if (row['apartment_features'] != ''):
            desc_1.append (str(row['apartment_features']).replace(' ','_').replace(',_',', ')\
            + ', ' + str(row['rooms'])+'_quarto'\
            + ', ' + str(row['rooms'])+'_quarto'\
            + ', ' + str(row['size'])+'_area'\
            + ', ' + str(row['size'])+'_area'\
            + ', ' + str(row['bathrooms'])+'_banheiro'\
            + ', ' + str(row['garage_spaces'])+'_vaga'\
            + ', ' + str(row['FAIXAS'])\
            + ', ' + str(row['FAIXAS'])\
            + ', ' + str(row['FAIXAS']))
            
        else:
            desc_1.append(str(row['rooms'])+'_quarto'\
            + ', ' + str(row['rooms'])+'_quarto'\
            + ', ' + str(row['size'])+'_area'\
            + ', ' + str(row['size'])+'_area'\
            + ', ' + str(row['bathrooms'])+'_banheiro'\
            + ', ' + str(row['garage_spaces'])+'_vaga'\
            + ', ' + str(row['FAIXAS'])\
            + ', ' + str(row['FAIXAS'])\
            + ', ' + str(row['FAIXAS']))

OLX1['DESC_3'] = desc_1

#### EFETUA LIMPEZA DAS FEATURE COM DESCRIÇÃO LIVRE E ARMAZENA NA DESC_4

In [12]:
import string

N_descs = []
N_descs2 = []

OLX1['desc'].fillna('')

for index, row in OLX1.iterrows():
    N_descs = [p for p in nltk.word_tokenize(str(row['desc']).lower())\
    if isinstance(p, float) == False\
    and p not in nltk.word_tokenize(row['title'].lower())\
    and p not in stop_words \
    and not p.isdigit()\
    and p not in string.punctuation\
    #and p not in nltk.word_tokenize(str(row['ENDERECO']).lower())\
    and p not in nltk.word_tokenize(str(row['apartment_features']).lower())\
    and p not in nltk.word_tokenize(str(row['apartment_complex_features']).lower())\
    and p not in nltk.word_tokenize(str(row['sellername']).lower())] 

    N_descs2.append( ', '.join(N_descs))

OLX1['DESC_4'] = N_descs2 


#### REMOÇÃO DE CARACTERES ESPECIAIS DAS DUAS BASES

In [13]:
#Carrega em uma lista

data = OLX1.DESC_3

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

################################################################################################################

data2 = OLX1.DESC_4

#Remove new line characters
data2 = [re.sub('\s+', ' ', sent) for sent in data2]

#### TOKEZAÇÃO

In [18]:
data_words=[]

for w in data: 
    data_words.append(w.split(','))

###############################################################################################################

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words2 = list(sent_to_words(data2))





#### CRIANDO FUNÇÕES DE BIGRAMAS, TRIGRAMAS STOP WORDS E LEMATIZAÇÃO

In [19]:

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

bigram = gensim.models.Phrases(data_words, min_count=4, threshold=1) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### APLICANDO FUNÇÕES DE BIGRAMAS E TRIGRAMAS STOP WORDS E LEMALIZAÇÃO NA BASE DE DESC LIVRE


In [20]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words2)

# Aplica bigrama
data_words_bigrams = make_bigrams(data_words_nostops)

data_words_bigrams = make_trigrams(data_words_nostops)


# pip install -U spacy
# python -m spacy download pt_core_news_sm
# Carrega Spacy em portugues
nlp = spacy.load("pt_core_news_sm")

# Executa lematização
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])




#### UNIFICA AS PALAVRAS DAS DUAS LISTAS

In [21]:
result_word = []
for i in range(0,len(data_words)):
    result_word.append (data_lemmatized[i]+data_words[i])    

#### CRIAÇÃO DE DICIONARIO CORPORA

In [22]:
id2word = corpora.Dictionary(result_word)

texts = result_word

corpus = [id2word.doc2bow(text) for text in texts]

#### VERIFICAÇÃO DE QUAL É A MELHOR QUANTIDADE DE TOPICOS

In [ ]:
%%time
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):

    from gensim.models.ldamodel import LdaModel
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=1, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=result_word, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Calcula o valor de coerencia de 2 a n
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=result_word, start=2, limit=40, step=1)

# Gera grafico
limit=40; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Gera dados de coerencia pela quantidade de topicos
for m, cv in zip(x, coherence_values):
    print("Numero Topicos =", m, " ;Valor de coerencia:", round(cv, 4))  #le 4 correspond au nombre de décimales du c

#### CRIAÇÃO DE MODELO LDA

In [24]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=12,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

#### DEMONSTRAÇÃO DOS TOPICOS

In [33]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"bairro" + 0.016*"paulo" + 0.012*"regiao" + 0.011*"santo" + '
  '0.009*"alto" + 0.009*"visto" + 0.009*"higienopolis" + 0.008*"tambem" + '
  '0.008*"localizar" + 0.008*"alar"'),
 (1,
  '0.034*" 1_quarto" + 0.022*" 1500-2000" + 0.021*" 2000-3000" + 0.019*" '
  '1_banheiro" + 0.014*" 0_vaga" + 0.012*"metro" + 0.012*"0" + 0.012*" 0" + '
  '0.011*"banheiro" + 0.011*" 1000-1500"'),
 (2,
  '0.017*"bairro" + 0.014*"morar" + 0.014*"paulista" + 0.012*"tambem" + '
  '0.011*"avenida" + 0.009*"restaurante" + 0.009*"proximo" + 0.008*"regiao" + '
  '0.008*"metro" + 0.008*"bom"'),
 (3,
  '0.018*" 2000-3000" + 0.017*" 3000-4000" + 0.016*"banheiro" + 0.015*" '
  '2_quarto" + 0.014*" 1_vaga" + 0.013*"cozinhar" + 0.013*" 1_quarto" + '
  '0.012*"metro" + 0.012*"vago" + 0.012*"codigo"'),
 (4,
  '0.018*" 1_quarto" + 0.017*" 2000-3000" + 0.013*"arouche" + 0.012*"espaco" + '
  '0.012*"bar" + 0.010*"piscina" + 0.010*"bairro" + 0.009*"incluso" + '
  '0.009*"galeria" + 0.009*" 1_banheiro"'),
 (5,
  

#### PERPLEXIDADE DO MODELO

In [34]:
# Perplexidade
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Coerencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=result_word, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.518145570450654

Coherence Score:  0.510575060393161


In [35]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis3 = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis3

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.118299 -0.112352       1        1  39.964717
1     -0.093002 -0.081623       2        1  30.674324
6     -0.200607  0.090766       3        1   9.345261
2      0.072170 -0.080237       4        1   7.596184
0      0.154317 -0.085307       5        1   6.110776
4     -0.057127  0.163190       6        1   3.534956
5      0.242548  0.105562       7        1   2.773783, topic_info=            Term         Freq        Total Category  logprob  loglift
3       1_quarto  2412.000000  2412.000000  Default  30.0000  30.0000
59        bairro  1045.000000  1045.000000  Default  29.0000  29.0000
52     2000-3000  2188.000000  2188.000000  Default  28.0000  28.0000
1306    gratuito   305.000000   305.000000  Default  27.0000  27.0000
1      1500-2000   930.000000   930.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
576        santo    14.683591   321.386992   Topic7  -5.4989   0.4990
357      centrar    14.332354   368.613752   Topic7  -5.5231   0.3377
147       parque    13.792348   261.591790   Topic7  -5.5615   0.6423
237     paulista    13.583764   546.039469   Topic7  -5.5767  -0.1089
52     2000-3000    13.236944  2188.301869   Topic7  -5.6026  -1.5229

[536 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         1  0.419824          0
0         2  0.526463          0
0         3  0.002245          0
0         4  0.011225          0
0         5  0.014593          0
...     ...       ...        ...
2031      6  0.897469   wireless
1010      4  0.330571          x
1010      5  0.264457          x
1010      7  0.363628          x
1012      7  0.876425  zoologico

[1395 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 7, 3, 1, 5, 6])

In [36]:
def Sort_Tuple(tup):   
    tup.sort(key = lambda x: x[-1])   
    return tup   

doc_num, topic_num, prob = [], [], []
print(lda_model.get_document_topics(corpus))
for n in range(len(OLX1)):
    get_document_topics = lda_model.get_document_topics(corpus[n])
    doc_num.append(n)
    sorted_doc_topics = Sort_Tuple(get_document_topics)
    topic_num.append(sorted_doc_topics[0][0])
    prob.append(sorted_doc_topics[0][1])
OLX1['Doc'] = doc_num
OLX1['Topic'] = topic_num
OLX1['Probability'] = prob


In [37]:
import ipywidgets as widgets
from IPython.display import display

#### FUNÇÃO DE RECOMENDAÇÃO

In [38]:

def recommend_by_storyline(n_id, df):
    recommended = []
    top10_list = []
   
    topic_num = df[df['ID']==n_id].Topic.values
    doc_num = df[df['ID']==n_id].Doc.values 
     
    output_df = df[df['Topic']==topic_num[0]].sort_values('Probability', ascending=False).reset_index(drop=True)
    index = output_df[output_df['Doc']==doc_num[0]].index[0]
 
    top10_list += list(output_df.iloc[index-15:index].index)
    top10_list += list(output_df.iloc[index+1:index+16].index)
 
    output_df['ID'] = output_df['ID'].str.title()
 
    for each in top10_list:
        recommended.append(output_df.iloc[each].ID)
 
    return recommended

In [39]:
OLX1_N = OLX1
colunas_1 = ['ID', 'TITULO', 'LINK', 'AREA', 'QUARTO', 'BANHEIRO','VAGA','DESC_1','DESC_2','IPTU','COND','ENDERECO','IMOBILIARIA','VALOR', 'CEP', 'FAIXAS', 'ORIGEM']
lista = ['list_id','title', 'link', 'size', 'rooms','bathrooms','garage_spaces','apartment_features', 'desc', 'iptu', 'condominio','sellername','price','zipcode', 'Doc', 'Topic', 'Probability']
OLX1_N = OLX1_N[lista]
OLX1_N = OLX1_N.rename(columns = {'list_id':"ID",'title':"TITULO", 'link':'LINK', 'size':"AREA", 'rooms':"QUARTOS",'bathrooms':"BANHEIROS",'garage_spaces':"VAGA",'apartment_features':"DESC_1",'desc':"DESC_2",'iptu':"IPTU", 'condominio':"COND",'sellername':"IMOBILIARIA",'price':"VALOR",'zipcode':"CEP"})

NID = []
for index, row in OLX1_N.iterrows():
    NID.append('id-'+ str(row['ID']))
OLX1_N['ID'] = NID
OLX1_N['PONT'] = 0
OLX1_N.head(1)


,ID,TITULO,LINK,AREA,QUARTOS,BANHEIROS,VAGA,DESC_1,DESC_2,IPTU,COND,IMOBILIARIA,VALOR,CEP,Doc,Topic,Probability,PONT
0,id-944660349,apartamento 1 dorm 1 vaga 44m2 campos eiiseos,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,44.0,1,1,1,0,codigo do anuncio ed5170 apartamento 1 dormito...,0,506.0,edge empreendimentos imobiliarios ltda,1350.0,1204001.0,0,2,0.038714,0


In [42]:
#print(recommend_by_storyline('id-919650400',OLX1_N))

In [43]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
global ids
ids2 = ''
doc_selec = ''
doc_list = []
for d in range(len(lda_model.print_topics())):
    doc_list.append(d)

    
if doc_selec == '':
    doc_selec = np.random.choice(doc_list)

    
ids = np.random.choice(OLX1_N[OLX1_N['Topic']==doc_selec].ID.values) 


OLX1_N2 = OLX1_N.loc[OLX1_N['ID'] == ids]

button1 = widgets.Button(description="CURTIR")
button2 = widgets.Button(description="PROXIMO")

output = widgets.Output()


display(button1,button2, output, OLX1_N2)
webbrowser.open(OLX1_N2['LINK'].values[0], new=2)


def on_button_clicked1(button1):
    with output:
        output.clear_output()
        global ids2
        if len(ids2) == 0:
            ids2 = ids
        
        OLX1_N['PONT'][OLX1_N['ID'] == str(ids2)] = OLX1_N['PONT'][OLX1_N['ID'] == str(ids2)]+1
        ids2 = str(np.random.choice(recommend_by_storyline(ids,OLX1_N)).lower())
        OLX1_N2 = OLX1_N.loc[OLX1_N['ID'] == ids2]
        print("Curtiu")   
        webbrowser.open(OLX1_N2['LINK'].values[0], new=2)
        display(OLX1_N2)
        
        

def on_button_clicked2(button2):
    with output:
        doc_selec2 = doc_selec
        output.clear_output()
        doc_list2 = list(filter(lambda a: a != doc_selec2, doc_list))
        doc_selec2 = np.random.choice(doc_list2)
        ids = np.random.choice(OLX1_N[OLX1_N['Topic']==doc_selec2].ID.values)
        OLX1_N2 = OLX1_N.loc[OLX1_N['ID'] == str(ids)]
        print("Não Curtiu " + str(doc_selec2))
        webbrowser.open(OLX1_N2['LINK'].values[0], new=2)
        display(OLX1_N2)
        
       
        
button1.on_click(on_button_clicked1)
button2.on_click(on_button_clicked2)

<img src=./img/app1.jpg width="1200" />